########### COULD TRY https://scikit-learn.org/dev/modules/feature_extraction.html

#### Should Try: Creating a sparse matrix

In [1]:
import sys
print sys.path
import general.scripts.modeling as mdl
import general.scripts.data_manipulation as manip
import general.scripts.transformers as transformers
import general.scripts.modeling as modeling
import general.scripts.uber_helpers as uber_helpers
import general.scripts.viz as viz

['', '/home/udocker/phoenix-worker/environments/python2/pyenv2', '/home/udocker/phoenix-worker/nbextensions', '/home/udocker/phoenix-worker/environments/python2/lib/python2.7', '/home/udocker/phoenix-worker/environments/python2/lib/python2.7/plat-x86_64-linux-gnu', '/home/udocker/phoenix-worker/environments/python2/lib/python2.7/lib-tk', '/home/udocker/phoenix-worker/environments/python2/lib/python2.7/lib-old', '/home/udocker/phoenix-worker/environments/python2/lib/python2.7/lib-dynload', '/usr/lib/python2.7', '/usr/lib/python2.7/plat-x86_64-linux-gnu', '/usr/lib/python2.7/lib-tk', '/home/udocker/phoenix-worker/environments/python2/local/lib/python2.7/site-packages', '/home/udocker/phoenix-worker/environments/python2/lib/python2.7/site-packages', '/home/udocker/phoenix-worker/environments/python2/local/lib/python2.7/site-packages/IPython/extensions', '/home/udocker/.ipython', '/home/udocker/phoenix-worker/environments/python2/local/lib/python2.7/site-packages/jaeger_client/thrift_gen']

2020-03-28 18:28:36,676 jaeger_tracing WARNING Jaeger tracer already initialized, skipping


In [2]:

# General python libs
import datetime
start = datetime.datetime.now()
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')  # prettier plots

from IPython.core.display import display, HTML

display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_columns', None)
# from tqdm import tqdm
import math

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import precision_recall_curve
from sklearn.utils.fixes import signature
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from xgboost import XGBClassifier, Booster
from xgboost import plot_importance
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_curve, average_precision_score, roc_curve, auc, f1_score
from sklearn.metrics import confusion_matrix
from xgboost import plot_importance
from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import pickle
import os
# Import Spark libs

2020-03-28 18:28:36,719 jaeger_tracing WARNING Jaeger tracer already initialized, skipping


In [3]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import FeatureUnion, Pipeline 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GroupShuffleSplit

In [4]:
# from tqdm.notebook import tqdm
# tqdm.pandas()

from tqdm.auto import tqdm
tqdm.pandas()

# Data

In [305]:
categorical_features
numerical_features
id_features

In [306]:
# Define the feature extractor pipelines
data_cleaning_pipeline = Pipeline([
        ('df_clean', transformers.BasicDFCleaning())
])

categorical_feature_clean_pipeline = Pipeline([
        ('categorical_feature_select', transformers.FeatureSelector(categorical_features)),
        # Prefix for each column in the missing values
        ('impute_missing_1', SimpleImputer(strategy='constant', fill_value='missing')),
        ('impute_missing_none', SimpleImputer(missing_values=None, strategy='constant', fill_value='missing')),
        ('makeDF', transformers.MakeDFFromNumpyPipeline(categorical_features)),
        ('clean_categorical', transformers.CategoricalCleaning())
])

numerical_feature_clean_pipeline = Pipeline([
        ('numerical_feature_select', transformers.FeatureSelector(numerical_features))
])


union_categ_numer_after_basic_clean = FeatureUnion( 
                                transformer_list = [ 
                                        ('id_features', transformers.FeatureSelector(id_features)),
                                        ( 'categorical_pipeline', categorical_feature_clean_pipeline ),
                                        ('numerical_pipeline', numerical_feature_clean_pipeline)
] )

# Categorical Column Pivot Value Vectorize
feat = 'categ_feature_1'
custom_feature_processing_categ_feature_1_pivot_time_diff = Pipeline([
        ('reqd_features', transformers.FeatureSelector(['entity_uuid', feat,'time_min'])),
        ('pivot_and_choose', transformers.GroupMultiCategoricalToPivotedColumnsWithExternalValue('entity_uuid', feat, 'time_min', True, 1000000000))
])

feat = 'categ_feature_2'
custom_feature_processing_categ_feature_2_pivot_time_diff = Pipeline([
        ('reqd_features', transformers.FeatureSelector(['entity_uuid', feat,'time_min'])),
        ('pivot_and_choose', transformers.GroupMultiCategoricalToPivotedColumnsWithExternalValue('entity_uuid', feat, 'time_min', True, 1000000000))
])

feat = 'categ_feature_3'
custom_feature_processing_categ_feature_3_pivot_time_diff = Pipeline([
        ('reqd_features', transformers.FeatureSelector(['entity_uuid', feat,'time_min'])),
        ('pivot_and_choose', transformers.GroupMultiCategoricalToPivotedColumnsWithExternalValue('entity_uuid', feat, 'time_min', True, 1000000000))
])

# Sklearn feature unions do not return column names or df, rather only return numpy arrays. Feature names can be later derived from the cutom vectorizers later using 'get_feature_names'
union_custom_feature_processing = FeatureUnion( 
                                transformer_list = [ 
                                        ('processed_categ_feature_1_pivot', custom_feature_processing_action_pivot_time_diff),
                                        ('processed_categ_feature_2_pivot', custom_feature_processing_error_key_pivot_time_diff),
                                        ('processed_categ_feature_3_pivot', custom_feature_processing_action_type_pivot_time_diff)
#                                         ( 'categorical_pipeline', categorical_feature_clean_pipeline ),
] )



feature_eng_pipeline = Pipeline([
        ('df_clean', data_cleaning_pipeline), # Clean the overall dataframe
        ('clean_categorical_and_numerical', union_categ_numer_after_basic_clean), # Run basic cleaning for both categoricals and numericals
        ('makeDF', transformers.MakeDFFromNumpyPipeline(id_features+categorical_features+numerical_features)), # Return a DF object instead of an ndarray
        ('run_custom_processing', union_custom_feature_processing),
])

# Count Vectorizers not used in the final pipeline
# ------------------------------------------------
# Categorical Column Count Vectorize (Restriction: Binary)
categ_feature_1_count_vectorize = Pipeline([
    ('feature_select', transformers.FeatureSelector(['entity_uuid','categ_feature_1'])),
    ('make_text', transformers.GroupByCreateText(['entity_uuid'],'categ_feature_1')),
    ('feature_select_action', transformers.FeatureSelector(['categ_feature_1'], True)),
    ('count_vectorize', CountVectorizer(binary=True))
])

# Categorical Column Count Vectorize 
categ_feature_1_count_vectorize = Pipeline([
    ('feature_select', transformers.FeatureSelector(['entity_uuid','categ_feature_1'])),
    ('make_text', transformers.GroupByCreateText(['entity_uuid'],'categ_feature_1')),
    ('feature_select_action', transformers.FeatureSelector(['categ_feature_1'], True)),
    ('count_vectorize', CountVectorizer(binary=False))
])



In [307]:
pipeline_get_labels = Pipeline([
    ('pick_one_label', transformers.GroupBySelectOne(['entity_uuid'],'label')),
    ('feature_select_label', transformers.FeatureSelector(['label'], True)),
])

### Create Test and Train Action Level grouped on contact

In [3]:
X = df_all_class.drop('label', axis = 1)
y = df_all_class['label'].values 

group_splitter = GroupShuffleSplit(n_splits=1, train_size=.8, random_state=123)
train_idx, test_idx = next(group_splitter.split(X, y, groups=df_all_class['entity_uuid']))

In [6]:
train = df_all_class.iloc[train_idx]
test = df_all_class.iloc[test_idx]

### Run Transformer Pipelines

In [7]:
train = feature_eng_pipeline.fit_transform(train)
test = feature_eng_pipeline.transform(test)

In [314]:
# Name the features
f1 = feature_eng_pipeline.named_steps['run_custom_processing'].transformer_list[0][1].named_steps['pivot_and_choose'].get_feature_names()
f2 = feature_eng_pipeline.named_steps['run_custom_processing'].transformer_list[1][1].named_steps['pivot_and_choose'].get_feature_names()
f3 = feature_eng_pipeline.named_steps['run_custom_processing'].transformer_list[2][1].named_steps['pivot_and_choose'].get_feature_names()

In [315]:
pipeline_name_features = Pipeline([
    ('makeDF_name_features', transformers.MakeDFFromNumpyPipeline(f1 + f2 + f3)), # Return a DF object instead of an ndarray
])
train_x = pipeline_name_features.fit_transform(train)
test_x = pipeline_name_features.fit_transform(test)

In [8]:
train_y = pipeline_get_labels.fit_transform(train)
test_y = pipeline_get_labels.fit_transform(test)

### Model (XGBoost, [action],all population)

In [319]:
xgbm = modeling.XGBoostModel(train_x, train_y, test_x, test_y, model_type='XGBoost')

In [9]:
xgbm.fit_predict_viz()

In [11]:
from xgboost import plot_importance
plot_importance(xgbm.model_obj, max_num_features=20, importance_type='gain')